# 5. Whale vs MEV — Comparison & Final Report

This notebook combines:

- **Whale analysis** (from Chapter 3)
- **MEV analysis** (from Chapter 4)
- A concise **final risk summary** and **narrative material**

It is designed as a *presentation-ready* notebook to:
- Compare **whale** and **MEV-like** addresses
- Summarize their behavior, structure and roles in the network
- Provide ready-to-use text and figures for reports or slides.


## 1. Setup & Data Loading

We reuse:

- The cleaned transaction table `tx`
- Whale detection results (Chapter 3)
- A lightweight MEV heuristic (built here for comparison)


In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["axes.grid"] = True

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(PROJECT_ROOT)

from src.data.load_data import load_clean_transactions

print("PROJECT_ROOT:", PROJECT_ROOT)

tx = load_clean_transactions()
tx['datetime'] = pd.to_datetime(tx['block_timestamp'])
print("Transactions:", len(tx))
tx.head()


PROJECT_ROOT: /Users/dada/Developer/italy_proj/DataMining/EhereumNetworkAnalysis
Transactions: 13268


,hash,from_address,to_address,block_number,value,block_timestamp,datetime
0,0xd8ec648861cf4de73f18f9a034623eeded1b26ec7246...,0xa9264494a92ced04747ac84fc9ca5a0b9549b491,0x835033bd90b943fa0d0f8e5382d9dc568d3fbd96,23772289,4.699994e+19,2025-11-11 00:00:11+00:00,2025-11-11 00:00:11+00:00
1,0x5843a9e865f9b7222ddb376ea2869c50b389c3a0d858...,0xc0ffeebabe5d496b2dde509f9fa189c25cf29671,0xc0ffeebabe5d496b2dde509f9fa189c25cf29671,23772292,5.817089e+19,2025-11-11 00:00:47+00:00,2025-11-11 00:00:47+00:00
2,0x131571aec26cd23b0134a97341acf9fb0b559b085b68...,0xe50008c1d110da8e56982f46a9188a292ee90a7b,0x1ab4973a48dc892cd9971ece8e01dcc7688f8f23,23772292,3.390013e+18,2025-11-11 00:00:47+00:00,2025-11-11 00:00:47+00:00
3,0xa1b7caf05dd498111a40ffe269fefb2ae574dde53da0...,0xe40d548eb4fa4d9188fd21723f2fd377456c0876,0x28c6c06298d514db089934071355e5743bf21d60,23772292,7.999922e+18,2025-11-11 00:00:47+00:00,2025-11-11 00:00:47+00:00
4,0xc1d8e4ffa9e7864d5a38f84aa4532308d411ba35f82e...,0x0eb1665de6473c624dcd087fdeee27418d65ed59,0xa03400e098f4421b34a3a44a1b4e571419517687,23772292,6.318854e+18,2025-11-11 00:00:47+00:00,2025-11-11 00:00:47+00:00


In [3]:
# Load whale detection results
INTERIM = os.path.join(PROJECT_ROOT, "data", "processed")
whale_path = os.path.join(INTERIM, "whale_detection_value_and_centrality.parquet")

if os.path.exists(whale_path):
    whales = pd.read_parquet(whale_path)
    print("Loaded whale table:", whales.shape)
else:
    print("WARNING: whale detection file not found:", whale_path)
    whales = pd.DataFrame()

whales.head()


Loaded whale table: (7796, 20)


,total_out_value,n_out_tx,total_in_value,n_in_tx,n_total_tx,net_flow,is_out_whale,is_in_whale,is_whale_value,in_degree,out_degree,degree,pagerank,hub_score,authority_score,is_whale_degree,is_whale_pagerank,is_whale_authority,is_whale_centrality,is_whale
0x0000000000000068f116a894984e2db1123eb395,0.0,0.0,9.491430e+19,15.0,15.0,9.491430e+19,False,False,False,14,0,14,0.000286,-0.000000,2.274776e-17,False,False,False,False,False
0x0000000000001ff3684f28c67538d4d072c22734,0.0,0.0,6.716182e+20,47.0,47.0,6.716182e+20,False,False,False,40,0,40,0.000513,-0.000000,6.715330e-04,False,False,False,False,False
0x0000000000a39bb272e79075ade125fd351887ac,0.0,0.0,1.401000e+20,23.0,23.0,1.401000e+20,False,False,False,20,0,20,0.000374,-0.000000,8.615999e-06,False,False,False,False,False
0x00000000219ab540356cbb839cbe05303d7705fa,0.0,0.0,1.853817e+22,347.0,347.0,1.853817e+22,False,False,False,311,0,311,0.006200,-0.000000,3.904840e-09,True,True,False,True,True
0x00000047bb99ea4d791bb749d970de71ee0b1a34,0.0,0.0,1.306812e+20,15.0,15.0,1.306812e+20,False,False,False,10,2,12,0.000195,0.000036,2.554334e-04,False,False,False,False,False


In [4]:
# Whale address set
if not whales.empty and "is_whale" in whales.columns:
    whale_addrs = whales.index[whales["is_whale"] == True]
    print("Number of whales:", len(whale_addrs))
else:
    whale_addrs = []
    print("No whale labels available.")


Number of whales: 21


### 1.1 MEV-like Address Heuristic

For MEV-like behavior, we use a simple heuristic based on:

- Same-block multi-tx bursts
- Calls to known DEX routers

This is sufficient for **relative comparison** with whales.


In [ ]:
# Placeholder router address list — update with real routers if available
# === Recreate router_type (same as in 4.5) ===

#### We reuse the **router address list** from 4.2.
router_addresses = {
    "uniswap_v2": [
        "0xf164fc0ec4e93095b804a4795bbe1e041497b92a",
    ],
    "uniswap_v3": [
        "0xe592427a0aece92de3edee1f18e0157c05861564",
    ],
    "1inch": [
        "0x1111111254eeb25477b68fb85ed929f73a960582",
    ],
    "0x_exchange": [
        "0xdef1c0ded9bec7f1a1670819833240f027b25eff",
    ],
    "paraswap": [
        "0xdef171fe48cf0115b1d80b88dc8eab59176fee57",
    ],
    "sushi_swap": [
        "0xd9e1ce17f2641f24ae83637ab66a2cca9c378b9f",
    ],
    "balancer": [
        "0xba12222222228d8ba445958a75a07044adaf5ab",
    ],
    "curve": [
        "0x11111112542d85b3ef69ae05771c22d518637fe",
    ],
    "kyber_swap": [
        "0x1c87257f5e8609940bc751a07bb5c9c3e1b0b76",
    ]
}

router_all = {a.lower() for lst in router_addresses.values() for a in lst}

tx["to_address_lower"] = tx["to_address"].str.lower()
tx["is_router_tx"] = tx["to_address_lower"].isin(router_all)

# Same-block multi-tx bursts per address
block_counts = (
    tx.groupby(["from_address", "block_number"])["hash"]
      .count()
      .rename("tx_in_block")
)

burst_stats = (
    block_counts.reset_index()
    .groupby("from_address")["tx_in_block"]
    .max()
    .rename("max_tx_in_block")
)

# Build simple MEV base table
mev_base = pd.DataFrame(index=tx["from_address"].dropna().unique())
mev_base["router_tx_count"] = (
    tx[tx["is_router_tx"]]
    .groupby("from_address")["hash"]
    .count()
)
mev_base["router_tx_count"] = mev_base["router_tx_count"].fillna(0)
mev_base = mev_base.join(burst_stats, how="left")
mev_base["max_tx_in_block"] = mev_base["max_tx_in_block"].fillna(0)

# Heuristic MEV-like label
mev_base["is_mev_like"] = (
    (mev_base["router_tx_count"] >= 1) &
    (mev_base["max_tx_in_block"] >= 2)
)

mev_like_addrs = mev_base.index[mev_base["is_mev_like"]].tolist()
print("Number of MEV-like addresses (heuristic):", len(mev_like_addrs))
mev_base.head()


Number of MEV-like addresses (heuristic): 0


,router_tx_count,max_tx_in_block,is_mev_like
0xa9264494a92ced04747ac84fc9ca5a0b9549b491,0.0,1,False
0xc0ffeebabe5d496b2dde509f9fa189c25cf29671,0.0,1,False
0xe50008c1d110da8e56982f46a9188a292ee90a7b,0.0,1,False
0xe40d548eb4fa4d9188fd21723f2fd377456c0876,0.0,1,False
0x0eb1665de6473c624dcd087fdeee27418d65ed59,0.0,1,False


## 2. Behavior Comparison: Whales vs MEV-like Addresses

We compare basic behavioral metrics:

- Number of transactions
- Active hours
- Router usage
- Same-block bundle behavior


In [ ]:
tx["hour"] = tx["datetime"].dt.hour

addr_activity = (
    tx.groupby("from_address")
      .agg(
          n_tx=("hash", "count"),
          active_hours=("hour", "nunique"),
          n_router_tx=("is_router_tx", "sum"),
      )
)

addr_activity["address_type"] = "other"

if len(whale_addrs) > 0:
    addr_activity.loc[addr_activity.index.isin(whale_addrs), "address_type"] = "whale"

if len(mev_like_addrs) > 0:
    addr_activity.loc[addr_activity.index.isin(mev_like_addrs), "address_type"] = "mev_like"

addr_activity["address_type"].value_counts()


In [ ]:
# Summary statistics by group
group_stats = (
    addr_activity.groupby("address_type")[["n_tx", "active_hours", "n_router_tx"]]
    .describe()
)
group_stats


In [ ]:
# Boxplot of transaction counts per group
import seaborn as sns

plt.figure(figsize=(8, 4))
sns.boxplot(data=addr_activity, x="address_type", y="n_tx")
plt.yscale("log")
plt.title("Number of transactions per address — whales vs MEV-like vs others")
plt.show()


In [ ]:
# Router usage vs total tx
plt.figure(figsize=(8, 6))
sample = addr_activity.sample(min(len(addr_activity), 2000), random_state=42)
sns.scatterplot(data=sample, x="n_tx", y="n_router_tx", hue="address_type", alpha=0.7)
plt.xscale("log")
plt.yscale("log")
plt.title("Router usage vs total tx — whales vs MEV-like vs others")
plt.show()


## 3. Structural Comparison (Graph-Based)

Here we can optionally join **centrality** and **community** metrics
exported from previous notebooks (whale & MEV graph analysis).

If centrality exports are available under `data/interim`, they will be used;
otherwise this section remains a template.


In [ ]:
centrality_path = os.path.join(INTERIM, "mev_centrality_scores.csv")

if os.path.exists(centrality_path):
    cent = pd.read_csv(centrality_path)
    cent = cent.set_index("address")
    print("Loaded centrality scores:", cent.shape)

    addr_activity = addr_activity.join(cent[["degree", "pagerank", "betweenness"]], how="left")
else:
    print("No centrality export found at:", centrality_path)


### 3.1 Degree Centrality by Address Type (if available)


In [ ]:
if {"degree", "address_type"}.issubset(addr_activity.columns):
    plt.figure(figsize=(8,4))
    sns.boxplot(data=addr_activity[addr_activity["degree"].notna()],
                x="address_type", y="degree")
    plt.yscale("log")
    plt.title("Degree centrality (log-scale) by address type")
    plt.show()
else:
    print("Centrality not available — skipping structural plots.")


## 4. Roles in the Network (Narrative)

Based on the previous analysis:

- **Whales**:
  - Control large flows of ETH and tokens
  - Tend to interact with exchanges, bridges, OTC-like hubs
  - Act as *liquidity sources* or *sinks* over longer time horizons

- **MEV-like addresses**:
  - Fire multiple transactions in the same block
  - Interact heavily with DEX routers
  - Implement arbitrage, sandwich and other short-term extraction strategies
  - Act as *execution agents* that re-route liquidity between pools

Together, they shape both the **macroscopic** (whale flows)
and **microscopic** (MEV execution) structure of the network.


## 5. Combined Risk Analysis (Text Draft)

This section provides draft text for a combined **whale + MEV risk** view.
You can copy and adapt this for your final written report.


### 5.1 Whale Risk

- Volume is highly concentrated in a small set of whale addresses  
- Their actions can trigger:
  - Large price swings
  - Liquidity relocation between venues
  - Deleveraging cascades if positions are collateralized  

Monitoring whale **net flows**, **accumulation vs distribution**,
and **interactions with key venues** is critical for systemic risk assessment.


### 5.2 MEV Risk

- MEV-like addresses extract value from regular users via:
  - Front-running and back-running
  - Sandwich attacks
  - Cross-DEX arbitrage
- They contribute to:
  - Gas price spikes
  - Congestion at sensitive times
  - Fragmentation of execution quality across users

While MEV bots may improve price alignment across venues,
they also **redistribute value** in favor of highly optimized actors.


### 5.3 Joint View

Whales and MEV-like addresses represent **two layers of on-chain risk**:

1. **Position risk (whales)** — who holds and moves large balances
2. **Execution risk (MEV)** — who controls how trades are executed in the mempool / blocks

A holistic risk framework should monitor **both dimensions**.


## 6. Blog-Style Summary (Ready-to-Use Text)

Below is a short, blog-style explanation of the findings:

> On-chain data reveals two very different kinds of large players.
> **Whales** are the slow giants: they move huge amounts of value,
> but do so relatively infrequently. Their decisions reshape the
> supply and demand of assets over longer time horizons.
>
> **MEV bots**, on the other hand, are hyperactive snipers.
> They fire multiple transactions in the same block, hop between
> DEX routers, and try to capture tiny price inconsistencies or
> user slippage opportunities. Their footprint is small in terms
> of total volume, but very large in terms of transaction count
> and market microstructure impact.
>
> Together, whales and MEV bots form the hidden backbone of on-chain
> market dynamics: one provides the mass of capital, the other
> continuously rearranges it in search of yield.


## 7. Appendix — Reusable Code Snippets


In [ ]:
def compute_same_block_bursts(tx: pd.DataFrame) -> pd.Series:
    """Compute max number of tx in the same block per address."""
    counts = (
        tx.groupby(["from_address", "block_number"])["hash"]
          .count()
          .rename("tx_in_block")
    )
    bursts = (
        counts.reset_index()
        .groupby("from_address")["tx_in_block"]
        .max()
        .rename("max_tx_in_block")
    )
    return bursts


In [ ]:
def attach_address_labels(tx: pd.DataFrame,
                          whale_addrs,
                          mev_like_addrs) -> pd.Series:
    """Label addresses as whale / mev_like / other."""
    labels = pd.Series("other", index=tx["from_address"].dropna().unique())
    if len(whale_addrs) > 0:
        labels.loc[labels.index.isin(whale_addrs)] = "whale"
    if len(mev_like_addrs) > 0:
        labels.loc[labels.index.isin(mev_like_addrs)] = "mev_like"
    return labels
